<a href="https://colab.research.google.com/github/mmdedavoodi/cats-dogs-classification-cnn/blob/main/Cats_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1IcAf8TmM2T7HB_jvb-cKela94_ZIPuqP

In [ ]:
!unzip /content/cats_and_dogs_filtered.zip

In [ ]:
import random

import matplotlib.pyplot as plt
import torch
import cv2
import torchvision.transforms as t
import numpy as np
from glob import glob
from torchvision.datasets import DatasetFolder
from torch import nn
from tqdm import tqdm_notebook as tqdm
import torchvision
from google.colab.patches import cv2_imshow

In [ ]:
file_names_train = glob('cats_and_dogs_filtered/train/*/*.jpg')

In [ ]:
file_names_train


In [ ]:
file_names_train[0].split("/")[-2]

In [ ]:

image = cv2.imread(file_names_train[3])
image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
print(image.shape)
plt.imshow(image)


In [ ]:
class Dataset_Cats_Dog(torch.utils.data.Dataset):
  def __init__(self , list_name , transform = None):
    self.list_name = list_name
    random.shuffle(self.list_name)
    self.transform = transform

  def __len__(self):
    return len(self.list_name)

  def __getitem__(self , idx):
    image = cv2.imread(self.list_name[idx])
    label = self.list_name[idx].split("/")[-2]
    if label == "dogs":
      label = 0
    else:
      label = 1

    if self.transform:
      image = self.transform(image)


    return image , label


classid = {1 : "cats" , 0 : "dogs"}

In [ ]:
file_names_test = glob('/content/cats_and_dogs_filtered/validation/*/*.jpg')


In [ ]:
transform_Train = t.Compose([
    t.Lambda(lambda x : cv2.cvtColor(x , cv2.COLOR_BGR2RGB)),
    t.ToTensor(),
    t.Resize((224,224)),
    t.RandomCrop((224,224) , padding = 20),
    t.RandomHorizontalFlip(p=0.5),
    t.RandomRotation(20),
    t.ColorJitter(contrast=0.6 , brightness=0.6 , saturation=0.6),
    t.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_Test = t.Compose([
    t.Lambda(lambda x : cv2.cvtColor(x , cv2.COLOR_BGR2RGB)),
    t.ToTensor(),
    t.Resize((224 , 224)),
    t.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
DataSetTrain = Dataset_Cats_Dog(file_names_train , transform=transform_Train)
DataSetTest = Dataset_Cats_Dog(file_names_test , transform=transform_Test)

In [ ]:
i = 10
image , label = DataSetTrain[i]
plt.imshow(image.permute(1,2,0))
print(label)

In [ ]:
i = 10
image , label = DataSetTest[i]
plt.imshow(image.permute(1,2,0))
print(label)

## Data Loader

In [ ]:
Dataloader_Train = torch.utils.data.DataLoader(DataSetTrain, 64 , shuffle=True , num_workers=2 , prefetch_factor = 2)

Dataloader_Test = torch.utils.data.DataLoader(DataSetTest, 64 , shuffle=False , num_workers=2 , prefetch_factor = 2)



 ## Model Architecture

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = torch.nn.Sequential(
    nn.BatchNorm2d(3),

    nn.Conv2d(3 , 32 , 3 , 1 , 1),
    nn.ReLU(),
    nn.Conv2d(32 , 64 , 3 , 1 , 1),
    nn.ReLU(),
    nn.MaxPool2d(4 , 4),
    nn.BatchNorm2d(64),


    nn.Conv2d(64 , 64 , 3 , 1 , 1),
    nn.ReLU(),
    nn.Conv2d(64 , 128 , 3 , 1 , 1),
    nn.ReLU(),
    nn.MaxPool2d(4, 4),
    nn.BatchNorm2d(128),


    nn.Conv2d(128 , 128 , 3 , 1 , 1),
    nn.ReLU(),
    nn.Conv2d(128 , 256 , 3 , 1 , 1),
    nn.ReLU(),
    nn.MaxPool2d(4, 4),
    nn.BatchNorm2d(256),


    nn.Flatten(),
    nn.Linear(2304 , 1),
)


model.to(device)

## Optimizer and Loss Function

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
model(DataSetTrain[1][0].to(device).unsqueeze(0))

In [ ]:
len(Dataloader_Test.dataset)

## Train Phase

In [ ]:
def train_Phase(model , dataloader , optimizer , loss_function , device):
  model.train()
  train_phase_loss = 0
  train_phase_acc = 0
  sample = len(dataloader.dataset)
  pbar = tqdm(dataloader , desc = "Training")
  for image , label in pbar:
    image , label = image.to(device) , label.to(device)
    label = label.view(-1, 1).float()
    output = model(image)
    output_sigmoid = torch.sigmoid(output)
    loss_value = loss_function(output , label)
    loss_value.backward()
    pbar.set_postfix_str(f"Loss = {loss_value:.3f}")
    optimizer.step()
    optimizer.zero_grad()

    train_phase_loss += loss_value.item() * len(label)
    train_phase_acc += (output_sigmoid.round() == label).sum().item()

  train_phase_loss /= sample
  train_phase_acc /= sample
  return train_phase_loss , train_phase_acc



In [ ]:
def test_Phase(model , dataloader , loss_function , device):
  model.eval()
  test_phase_loss = 0
  test_phase_acc = 0
  pbar = tqdm(dataloader , desc = "Testing")
  sample = len(dataloader.dataset)

  for image, label in pbar:
    image , label = image.to(device) , label.to(device)
    label = label.view(-1, 1).float()
    with torch.no_grad():
      output = model(image)
      output_sigmoid = torch.sigmoid(output)
      loss_value = loss_function(output , label)
      pbar.set_postfix_str(f"Loss = {loss_value:.3f}")

      test_phase_loss += loss_value.item() * len(label)
      test_phase_acc += (output_sigmoid.round() == label).sum().item()

  test_phase_loss /= sample
  test_phase_acc /= sample

  return test_phase_loss , test_phase_acc

In [ ]:
def run_epoch(model , train_dataloader , test_dataloader , optimizer , loss_function , device):
  train_loss , train_acc = train_Phase(model , train_dataloader , optimizer , loss_function , device)
  test_loss , test_acc = test_Phase(model , test_dataloader , loss_function , device)
  return train_loss , train_acc , test_loss , test_acc

In [ ]:
def train_model(model , train_dataloader , test_dataloader , optimizer , loss_function , device , epochs = 10):
  train_loss = []
  train_acc = []
  test_loss = []
  test_acc = []
  for epoch in range(epochs):
    try:
      train_phase_loss , train_phase_acc , test_phase_loss , test_phase_acc = run_epoch(model , train_dataloader , test_dataloader , optimizer , loss_function , device)
      print(f"Epoch {epoch + 1}: train loss {train_phase_loss:4f}, train acc {train_phase_acc * 100:2f} | test loss {test_phase_loss:4f}, test acc {test_phase_acc * 100:2f}")
      train_loss.append(train_phase_loss)
      train_acc.append(train_phase_acc)
      test_loss.append(test_phase_loss)
      test_acc.append(test_phase_acc)
    except KeyboardInterrupt:
      break
  return train_loss , train_acc , test_loss , test_acc

In [ ]:
def draw_plot(train_loss , train_acc , test_loss , test_acc):
  plt.figure(figsize=(10, 4))
  plt.subplot(1, 2, 1)
  plt.plot(train_loss, label="Train")
  plt.plot(test_loss, label="Test")
  plt.legend()
  plt.grid()
  plt.ylabel('Loss')
  plt.xlabel('Epoch')

  plt.subplot(1, 2, 2)
  plt.plot(train_acc, label="Train")
  plt.plot(test_acc, label="Test")
  plt.legend()
  plt.grid()
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')

  return plt

## Run Model

In [ ]:
train_loss , train_acc , test_loss , test_acc = train_model(model , Dataloader_Train , Dataloader_Test , optimizer , loss_function , device , 30)


In [ ]:
draw_plot(train_loss , train_acc , test_loss , test_acc)